In [ ]:
!ls ../input/tf-installing-packageofficial-model-and-text

In [ ]:
!mkdir -p /tmp/pip/cache/

In [ ]:
!cp ../input/tf-installing-packageofficial-model-and-text/cached_property-1.5.2-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/gast-0.4.0-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/gin_config-0.4.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/grpcio-1.34.1-cp37-cp37m-manylinux2014_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/portalocker-2.0.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/py-cpuinfo-8.0.0.xyz.csv /tmp/pip/cache/py-cpuinfo-8.0.0.tar.gz
!cp ../input/tf-installing-packageofficial-model-and-text/pycocotools-2.0.2.xyz.csv /tmp/pip/cache/pycocotools-2.0.2.tar.gz
!cp ../input/tf-installing-packageofficial-model-and-text/sacrebleu-1.5.1-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/seqeval-1.2.2-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorboard-2.5.0-py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_estimator-2.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_model_optimization-0.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tensorflow_text-2.5.0-cp37-cp37m-manylinux1_x86_64.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tf_models_official-2.5.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../input/tf-installing-packageofficial-model-and-text/tf_slim-1.1.0-py2.py3-none-any.whl /tmp/pip/cache/


In [ ]:
!mkdir -p /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ tensorflow-text
!pip install --no-index --find-links /tmp/pip/cache/ tf-models-official

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
from tqdm import tqdm
import os
import glob
import datetime
import json
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [ ]:
#tfhub_handle_encoder = "/kaggle/input/bertpreprocessing-v3-en-uncased-l12-v4/small_bert_bert_en_uncased_L-8_H-512_A-8_2"
#tfhub_handle_preprocess = "/kaggle/input/bertpreprocessing-v3-en-uncased-l12-v4/bert_en_uncased_preprocess_3"
#train_path = "/kaggle/input/coleridgeinitiative-show-us-the-data/train/"
test_path = "/kaggle/input/coleridgeinitiative-show-us-the-data/test/"

# Get Columns Name

In [ ]:
df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
df = df.drop(['pub_title','dataset_title','dataset_label'], axis =1)

In [ ]:
labelencoder = LabelEncoder()
df['cleaned_label'] = labelencoder.fit_transform(df['cleaned_label'])
ohencoder = OneHotEncoder()
onehot_label = df.iloc[:,-1].values
onehot_label = onehot_label.reshape(-1,1)
onehot_label = ohencoder.fit_transform(onehot_label)
onehot_label = pd.DataFrame(onehot_label.todense())

In [ ]:
onehot_label = pd.DataFrame(onehot_label)
dataset_resource = list()
for col in onehot_label.columns:
    dataset_resource.append(col)
dataset_resource = labelencoder.inverse_transform(dataset_resource)

# Making Predictions

Get Testing Text

In [ ]:
test_file_list = !ls ../input/coleridgeinitiative-show-us-the-data/test
test_file_list = [id.strip('.json') for id in test_file_list]

In [ ]:
def clean_text(text):
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)#remove url
    text = re.sub(r"\S+@\S+", ' ', text) #remove email address
    text = re.sub(r"[^a-zA-Z.!?']", ' ', text) #remove special symbol
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
def read_test_json(test_file_list, path): #Get Training Text
    file_list = []
    text_data = []
    for i, rec_id in tqdm(enumerate(test_file_list), total = len(test_file_list)):
        location = f'{path}{rec_id}.json'
        file_list.append(rec_id)
        add_text = str("")
        with open(location, 'r') as f:
            text_data.append(json.load(f))
    for j in range(len(text_data)):
        temp_dict = []
        add_text = str("")
        temp_dict = text_data[j]
        for k in range(1): #only get abstract
            temp_text_iter = dict(temp_dict[k])
            add_text= add_text + temp_text_iter['text']    
        text_data[j] = add_text
        text_data[j] = clean_text(text_data[j])
        
    text_df = pd.DataFrame(list(zip(file_list, text_data)),
                           columns =['ID', 'text'])
    
    return text_df

In [ ]:
text_df_test = read_test_json(test_file_list, test_path)

In [ ]:
classifier_model = tf.saved_model.load('../input/my-pre-trained-bert-model/temp_001')


In [ ]:
def get_prediction(text):
    result = tf.constant([text])
    result = classifier_model(result)
    result = tf.sigmoid(result)
    result = result.numpy()
    result = result[0][:]
    
    return result

In [ ]:
y_sub = text_df_test["text"].apply(lambda x: get_prediction(x))
y_sub

In [ ]:
for i in range(y_sub.shape[0]):
    
    a = y_sub[i]
    a = a >= 0.05
    if True not in a:
        a = y_sub[i]
        a = a.max() == a
    
    y_sub[i] = a
    
y_sub

In [ ]:
label_sub = []
for i in range(y_sub.shape[0]):
    temp_label = []
    temp_text = str('')
    for j in range(y_sub[0].shape[0]):
        if y_sub[i][j] == True:
            temp_label = dataset_resource[j]
            temp_text = str(temp_text+'|'+temp_label)
            temp_text = temp_text.strip()  
            temp_text = temp_text.strip('|')
    label_sub.append(temp_text)

In [ ]:
submission = pd.DataFrame(list(zip(test_file_list,
                                   label_sub)),
                          columns =['Id', 'PredictionString'])

In [ ]:
submission

In [ ]:
submission.iloc[3,1]

In [ ]:
submission.to_csv('submission.csv',index=False)
